In [1]:
import pickle
import torch
import pickle
import pandas as pd
import csv
from helper.model import Net
from helper.utils import *
from helper.distance_map import *
from helper.evaluate import *
from eval import *
import numpy as np
dtype = torch.float32
device = torch.device("cuda")

### Use maximum separation to call EC

In [22]:
file_name = "./data/uniref10_test_split_1_curate"
result = open(file_name+'.csv','r')
csvreader = csv.reader(result, delimiter = '\t')
all_label = set()
true_label_dict = {}
header = True
count = 0
for row in csvreader:
    # don't read the header
    if header is False:
        count += 1
        true_label_dict[row[0]] = row[1].split(';')
        for ec in row[1].split(';'):
            all_label.add(ec)
    if header: header = False
    

print(true_label_dict)

{'C7C422': ['3.5.2.6'], 'Q570B4': ['2.3.1.199'], 'Q6H9K0': ['3.2.1.67'], 'Q9Y1J3': ['2.7.11.1'], 'Q27651': ['2.7.1.11'], 'Q8W1X2': ['2.7.1.35'], 'Q79FW5': ['5.4.2.12'], 'Q9UBM1': ['2.1.1.17', '2.1.1.71'], 'Q9HCN3': ['3.1.1.4'], 'Q6L8Q7': ['3.1.13.4'], 'P12019': ['3.1.4.35', '3.1.4.53'], 'Q23917': ['3.1.4.53'], 'Q8I0P7': ['3.1.4.17'], 'P0A6E9': ['6.3.3.3'], 'Q8RCP4': ['6.3.5.11'], 'Q466A2': ['2.5.1.147'], 'P48368': ['2.5.1.29'], 'Q2A1A1': ['4.2.1.33'], 'Q86WU2': ['1.1.2.4'], 'Q631X2': ['1.1.1.27'], 'A6U3S5': ['5.3.1.26'], 'P81664': ['2.7.1.19'], 'Q9RTY9': ['4.2.1.33'], 'P23010': ['2.7.1.19'], 'Q7SA85': ['3.4.21.53'], 'A5FS18': ['1.1.1.37'], 'A9W565': ['3.4.23.36'], 'A9BHW9': ['2.5.1.75'], 'B7IHN7': ['2.4.1.227'], 'Q15VK3': ['2.7.7.18'], 'Q2KYV6': ['2.7.7.18'], 'Q0T900': ['5.1.3.24'], 'P35338': ['3.2.1.67'], 'P85439': ['1.11.1.7'], 'P85346': ['1.11.1.7'], 'Q9LW07': ['3.2.1.15'], 'P86054': ['1.11.1.7'], 'Q8ZWK6': ['2.7.2.3'], 'Q0U6Q5': ['2.3.1.1'], 'Q00359': ['3.2.1.67'], 'O27911': ['2.7.

In [12]:
def get_true_labels(file_name: str, entries: list):
    all_label = set()
    true_label_dict = {}
    result = open(file_name+'.csv','r')
    csvreader = csv.reader(result, delimiter = '\t')
    entries_set = set(entries)
    for rows in csvreader:
        if rows[0] in entries_set:
            true_label_dict[rows[0]] = rows[1].split(';')
            for ec in rows[1].split(';'):
                all_label.add(ec)
 
    true_label = []
    for i in entries:
        true_label.append(true_label_dict[i])
    return true_label, all_label

In [3]:
args_model_name = "uniref10_train_split_1_3200"
args_test_data = "uniref10_test_split_1_curate"
args_train_data = "uniref10_train_split_1"

args_hidden_dim = 512
args_out_dim = 128
id_ec_train, ec_id_dict_train = get_ec_id_dict(
    './data/' + args_train_data + '.csv')
id_ec_test, ec_id_dict_test = get_ec_id_dict(
    './data/' + args_test_data + '.csv')

model = Net(args_hidden_dim, args_out_dim, device, dtype)
checkpoint = torch.load('./model/uniref10_train_split_1_3200.pth')
model.load_state_dict(checkpoint)

emb_train = model(esm_embedding(ec_id_dict_train, device, dtype))
emb_test = model_embedding_test(id_ec_test, model, device, dtype)
eval_dist = get_dist_map_test(
    emb_train, emb_test, ec_id_dict_train, id_ec_test, device, dtype)
eval_df = pd.DataFrame.from_dict(eval_dist)
#eval_df.to_csv('./eval/distmap_' + args_test_data + '.csv')
out_filename = './eval/' + args_test_data
all_test_EC = write_top10_choices(eval_df, out_filename)
## maximum separation results
write_max_sep_choices(eval_df, out_filename, first_grad=False, use_max_grad=False)

The embedding sizes for train and test: torch.Size([25584, 128]) torch.Size([5492, 128])


100%|██████████| 4223/4223 [00:00<00:00, 16242.19it/s]


Calculating eval distance map, between 5492 test ids and 4223 train EC cluster centers


5492it [00:04, 1315.45it/s]


#### Use random nk samples to call EC

In [3]:
args_model_name = "uniref10_train_split_1_3200"
args_test_data = "uniref10_test_split_1_curate"
args_train_data = "uniref10_train_split_1"

args_hidden_dim = 512
args_out_dim = 128
id_ec_train, ec_id_dict_train = get_ec_id_dict(
    './data/' + args_train_data + '.csv')
id_ec_test, ec_id_dict_test = get_ec_id_dict(
    './data/' + args_test_data + '.csv')

model = Net(args_hidden_dim, args_out_dim, device, dtype)
checkpoint = torch.load('./model/uniref10_train_split_1_3200.pth')
model.load_state_dict(checkpoint)

emb_train = model(esm_embedding(ec_id_dict_train, device, dtype))
emb_test = model_embedding_test(id_ec_test, model, device, dtype)
eval_dist = get_dist_map_test(
    emb_train, emb_test, ec_id_dict_train, id_ec_test, device, dtype)
eval_df = pd.DataFrame.from_dict(eval_dist)
#eval_df.to_csv('./eval/distmap_' + args_test_data + '.csv')
out_filename = './eval/' + args_test_data
all_test_EC = write_top10_choices(eval_df, out_filename)
seed_everything()
rand_nk_ids, rand_nk_emb_train = random_nk_model(
    id_ec_train, emb_train, n=2, weighted=False)
random_nk_dist_map = get_random_nk_dist_map(
    emb_train, rand_nk_emb_train, ec_id_dict_train, rand_nk_ids, device, dtype)
write_random_nk_choices(
    eval_df, out_filename, random_nk_dist_map, p_value=0.001)


The embedding sizes for train and test: torch.Size([25584, 128]) torch.Size([5492, 128])


100%|██████████| 4223/4223 [00:00<00:00, 16858.40it/s]


Calculating eval distance map, between 5492 test ids and 4223 train EC cluster centers


5492it [00:04, 1335.60it/s]
100%|██████████| 4223/4223 [00:00<00:00, 17131.76it/s]
2000it [00:01, 1372.68it/s]
100%|██████████| 5492/5492 [00:15<00:00, 349.46it/s]


#### Training symmetric distance map refactor

In [ ]:
args_model_name = "uniref10_train_split_0"
args_hidden_dim = 512
args_out_dim  = 128
id_ec, ec_id_dict = get_ec_id_dict('./data/' + args_model_name + '.csv')
ec_id = {key: list(ec_id_dict[key]) for key in ec_id_dict.keys()}
dist_map = pickle.load( open('./data/distance_map/' + args_model_name + '.pkl', 'rb'))
model = Net(args_hidden_dim, args_out_dim, device, dtype)
checkpoint = torch.load('./model/uniref10_train_split_0_50.pth')
model.load_state_dict(checkpoint)
esm_emb = esm_embedding(ec_id_dict, device, dtype)

In [ ]:
args_train_file = "uniref10_train_split_0"
_, ec_id_dict = get_ec_id_dict('./data/' + args_train_file + '.csv')
# use cpu and high precision by default
device, dtype = torch.device("cpu"), torch.float64
esm_emb = esm_embedding(ec_id_dict, device, dtype)
esm_dist = get_dist_map(ec_id_dict, esm_emb, device, dtype)
esm_df = pd.DataFrame.from_dict(esm_dist)
# pickle.dump(esm_dist, open('./data/distance_map/' +
#             args_train_file + '.pkl', 'wb'))
esm_df.to_csv('./data/distance_map/' + args_train_file + '_df.csv')